This file should run in python3 envrionment

In [1]:
import os
import time
import sys

sys.path.append(os.path.abspath("../common"))
import pynq
import lpcv_eval

team_name = 'spring'
team = lpcv_eval.Team(team_name, batch_size = 1)

Import your processing code

In [2]:
import utils

Run all process

In [3]:
import cv2

interval_time = 0
total_time = 0
total_energy = 0

team.reset_batch_count()
rails = pynq.get_rails()

In [4]:
# Create detector with bit file name and model name
# you can replace the default bit file by your own bit
detector = utils.Processor(bit_name = "dpu.bit", model_name = "yolox.xmodel")
os.system('echo 1 > /proc/sys/vm/drop_caches')


start = time.time() 
# 5V power rail is used as an example for full sytem power modify the code below
# refer to ultra96_pmbus.ipynb notebook under 
# /home/xilinx/jupyter_notebooks/common/ultra96_pmbus.ipynb for details on power rails
recorder = pynq.DataRecorder(rails["5V"].power) 
all_results = list()
image_ids = []
img_list = [str(path) for path in team.img_list]
with recorder.record(0.05):
    all_results = detector.run(img_list)  
    #print("batch result size:", len(batch_result))

# timer stop after batch processing is complete
end = time.time()
t = end - start

total_time = t

#print('All processing time: {} seconds.'.format(total_time))
print("all results size:", len(all_results))
#print(all_results)
# Energy measurements    
energy = recorder.frame["5V_power"].mean() * t    
total_energy = energy
print("Total time:", total_time, "seconds")
print("Total energy:", total_energy, "J")

all results size: 20288
Total time: 495.29141426086426 seconds
Total energy: 2474.944665136189 J


In [5]:
mapping = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 28, 31, 32, 33, 34,
     35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63,
     64, 65, 67, 70, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 84, 85, 86, 87, 88, 89, 90]

In [6]:
# YOLOX save result
save_results = []
for i in range(len(all_results)):
    if all_results[i] is None:
        continue
    for j in range(len(all_results[i])):
        label = mapping[int(all_results[i][j][6])]
        score = all_results[i][j][5]
        x = all_results[i][j][1] #- all_results[i][j][3] / 2
        y = all_results[i][j][2] #- all_results[i][j][4] / 2
        width = all_results[i][j][3] - all_results[i][j][1]
        height = all_results[i][j][4] - all_results[i][j][2]
        #print("result[", i, "][", j, "] name:", all_images[i][0], "label:", label, ", score:", score, ", bbox:", x, y, width, height)
        image_id = img_list[i].split('/')[-1].split('.')[0]#image_id = str(all_images[i][0]).split('/')[-1].split('.')[0]
        save_results.append([image_id, x, y, width, height, label, score])
        #print([image_id, x, y, width, height, label, score])
# team.save_results_xml(save_results, total_time, total_energy)
# save results to json file for test accuracy
team.save_results_json(save_results)
print("save result done!")

save result done!
